In [1]:
import pandas as pd
import numpy as np

In [2]:
austin_outcomes = pd.read_csv("Data/Austin_Animal_Center_Outcomes.csv")
austin_intakes = pd.read_csv("Data/Austin_Animal_Center_Intakes.csv")

In [22]:
bloomington = pd.read_csv("Data/Bloomington_Animal_Shelter_Animals.csv")
louisville = pd.read_csv("Data/Louisville_Animal_IO_Data_5.csv")

In [ ]:
dallas14 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2014.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas15 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2015.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas16 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2016.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas17 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2017.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas18 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2018.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas19 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2019.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas20 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2020.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})

## Ongoing Data Wrangling To Do List

- remove animals that aren't cats and dogs
- remove all rows without outcomes
- standardize column names across locations
- add location columns for each dataframe
- make sure animal IDs don't overlap with others, might need to merge it with their location for unique IDs
- format Dates, Times, length of stay, age across each dataset
- merge Dallas together into one dataframe
- plot years of data to determine if there's enough data from the first recorded years in each location
- standardize categorical intake and outcomes entries across each location

### Austin Dataframe Formatting

In [3]:
austin_intakes.drop(['Name', 'MonthYear', 'Found Location'],axis=1, inplace=True)

austin_intakes.rename(columns={'Animal ID':'animal_id',
                      'Animal Type':'animal_type',
                      'DateTime':'intake_date',
                      'Intake Type':'intake_type',
                      'Intake Condition':'intake_condition',
                      'Breed':'breed',
                      'Color':'color',
                      'Sex upon Intake':'intake_sex',
                      'Age upon Intake':'intake_age'}, inplace=True)

In [4]:
austin_outcomes.drop(['Name', 'MonthYear'],axis=1, inplace=True)
austin_outcomes.rename(columns={'Animal ID':'animal_id',
                      'Animal Type':'animal_type',
                      'DateTime':'outcome_date',
                      'Outcome Type':'outcome_type',
                      'Outcome Subtype':'outcome_subtype',
                      'Breed':'breed',
                      'Color':'color',
                      'Sex upon Outcome':'outcome_sex',
                      'Age upon Outcome':'outcome_age',
                      'Date of Birth':'birth_date'}, inplace=True)

In [27]:
austin = austin_intakes.merge(austin_outcomes, how='left')
austin.head()

,animal_id,intake_date,intake_type,intake_condition,animal_type,intake_sex,intake_age,breed,color,outcome_date,birth_date,outcome_type,outcome_subtype,outcome_sex,outcome_age
0,A786884,01/03/2019 04:19:00 PM,Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,01/08/2019 03:11:00 PM,01/03/2017,Transfer,Partner,Neutered Male,2 years
1,A706918,07/05/2015 12:59:00 PM,Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,07/05/2015 03:13:00 PM,07/05/2007,Return to Owner,NaN,Spayed Female,8 years
2,A724273,04/14/2016 06:43:00 PM,Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,04/21/2016 05:17:00 PM,04/17/2015,Return to Owner,NaN,Neutered Male,1 year
3,A665644,10/21/2013 07:59:00 AM,Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,10/21/2013 11:39:00 AM,09/21/2013,Transfer,Partner,Intact Female,4 weeks
4,A682524,06/29/2014 10:38:00 AM,Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,07/02/2014 02:16:00 PM,06/29/2010,Return to Owner,NaN,Neutered Male,4 years


In [28]:
# split intake_sex into gender and intake_repro
austin[['intake_repro','gender']]= austin['intake_sex'].str.split(" ", expand=True)
# split outcome_sex into gender and outcome_repro
austin[['outcome_repro','gender']]= austin['outcome_sex'].str.split(" ", expand=True)

# set the new city variable to Austin
austin['city'] = 'Austin'

# after creating the new gender and repro columns, drop the old columns
austin.drop(['intake_sex','outcome_sex'], axis=1, inplace=True)

# subset to only Dogs
austin.drop(austin.loc[austin['animal_type']!='Dog'].index, inplace=True)

# fill in any animals still in the system to a TBD outcome
austin['outcome_type'] = austin['outcome_type'].fillna("TBD")

# set all string columns to lower case for consistency across all datasets
austin = austin.applymap(lambda s: s.lower() if type(s) == str else s)


austin.head()

,animal_id,intake_date,intake_type,intake_condition,animal_type,intake_age,breed,color,outcome_date,birth_date,outcome_type,outcome_subtype,outcome_age,intake_repro,gender,outcome_repro,city
0,A786884,01/03/2019 04:19:00 PM,Stray,Normal,Dog,2 years,Beagle Mix,Tricolor,01/08/2019 03:11:00 PM,01/03/2017,Transfer,Partner,2 years,Neutered,Male,Neutered,Austin
1,A706918,07/05/2015 12:59:00 PM,Stray,Normal,Dog,8 years,English Springer Spaniel,White/Liver,07/05/2015 03:13:00 PM,07/05/2007,Return to Owner,NaN,8 years,Spayed,Female,Spayed,Austin
2,A724273,04/14/2016 06:43:00 PM,Stray,Normal,Dog,11 months,Basenji Mix,Sable/White,04/21/2016 05:17:00 PM,04/17/2015,Return to Owner,NaN,1 year,Intact,Male,Neutered,Austin
4,A682524,06/29/2014 10:38:00 AM,Stray,Normal,Dog,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,07/02/2014 02:16:00 PM,06/29/2010,Return to Owner,NaN,4 years,Neutered,Male,Neutered,Austin
5,A743852,02/18/2017 12:46:00 PM,Owner Surrender,Normal,Dog,2 years,Labrador Retriever Mix,Chocolate,02/21/2017 05:44:00 PM,02/18/2015,Return to Owner,NaN,2 years,Neutered,Male,Neutered,Austin


In [30]:
# transform the single column of date and time into a column for each
# for both intake and outcome

austin['intake_time'] = pd.to_datetime(austin['intake_date']).dt.time
austin['intake_date'] = pd.to_datetime(austin['intake_date']).dt.date

austin['outcome_time'] = pd.to_datetime(austin['outcome_date']).dt.time
austin['outcome_date'] = pd.to_datetime(austin['outcome_date']).dt.date



In [31]:
# create a column for the difference between outcome and intake
# representing how long the animal has been in the system

austin.loc[:,'time_in'] = austin.loc[:,'outcome_date'] - austin.loc[:,'intake_date']

In [32]:
# to correctly track age, split the number and length of measurement into two columns
austin[['intake_age_n', 'intake_age_t']] = austin['intake_age'].str.split(" ",1,expand=True)

# set the number to int for calculating
austin['intake_age_n'] = austin['intake_age_n'].astype('int')

# when age is listed in years, months, or weeks, transform into days
i=0
while i < len(austin['intake_age_n']):
    if austin.iloc[i,-1] in ['year', 'years']:
        austin.iloc[i,-2] = austin.iloc[i,-2] * 365
    elif austin.iloc[i,-1] in ['month', 'months']:
        austin.iloc[i,-2] = austin.iloc[i,-2] * 30
    elif austin.iloc[i,-1] in ['week', 'weeks']:
        austin.iloc[i,-2] = austin.iloc[i,-2] * 7    
    i+=1

# drop the now unnecessary text portion of the original age column and the original column itself    
austin.drop(['intake_age_t', 'intake_age'], axis=1, inplace=True)

### Louisville Dataframe Formatting

In [23]:
louisville.drop(['SecondaryColor','IntakeReason','IntakeInternalStatus', 'OutcomeReason','OutcomeInternalStatus'], axis=1, inplace=True)

In [24]:
louisville.rename(columns={'AnimalID':'animal_id',
                          'AnimalType':'animal_type',
                          'IntakeDate':'intake_date',
                          'IntakeType':'intake_type',
                          'IntakeSubtype':'intake_subtype',
                          'PrimaryColor':'color',
                          'PrimaryBreed':'breed',
                          'SecondaryBreed':'breed_2',
                          'Gender':'gender',
                          'DOB':'birth_date',
                          'IntakeAsilomarStatus':'intake_condition',
                          'ReproductiveStatusAtIntake':'intake_repro',
                          'OutcomeDate':'outcome_date',
                          'OutcomeType':'outcome_type',
                          'OutcomeSubtype':'outcome_subtype',
                          'OutcomeAsilomarStatus':'outcome_condition',
                          'ReproductiveStatusAtOutcome':'outcome_repro',
                          }, inplace=True)
louisville.head()

,animal_id,animal_type,intake_date,intake_type,intake_subtype,color,breed,breed_2,gender,birth_date,intake_condition,intake_repro,outcome_date,outcome_type,outcome_subtype,outcome_condition,outcome_repro
0,A366370,CAT,2008-11-07 10:50:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,NaN,HEALTHY,ALTERED,2008-11-12 15:46:00,EUTH,FERAL,UNHEALTHY/UNTREATABLE,ALTERED
1,A366531,CAT,2008-11-10 10:20:00,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,HEALTHY,UNKNOWN,2008-11-19 20:10:00,EUTH,CONTAG DIS,HEALTHY,UNKNOWN
2,A532367,BIRD,2014-07-23 23:21:00,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,NaN,HEALTHY,FERTILE,2014-11-05 15:49:00,TRANSFER,NaN,HEALTHY,FERTILE
3,A532474,OTHER,2014-07-24 18:29:00,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,HEALTHY,UNKNOWN,2014-07-24 23:59:00,EUTH,MEDICAL,HEALTHY,UNKNOWN
4,A281756,DOG,2006-09-11 18:10:00,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,2005-09-11 00:00:00,HEALTHY,FERTILE,2006-09-12 13:44:00,EUTH,TIME/SPACE,HEALTHY,FERTILE


In [25]:
louisville.drop(louisville.loc[louisville['animal_type']!='DOG'].index, inplace=True)

louisville = louisville.applymap(lambda s: s.lower() if type(s) == str else s)

In [26]:
# transform the single column of date and time into a column for each
# for both intake and outcome

louisville['intake_time'] = pd.to_datetime(louisville['intake_date']).dt.time
louisville['intake_date'] = pd.to_datetime(louisville['intake_date']).dt.date

louisville['outcome_time'] = pd.to_datetime(louisville['outcome_date']).dt.time
louisville['outcome_date'] = pd.to_datetime(louisville['outcome_date']).dt.date

louisville['birth_date'] = pd.to_datetime(louisville['birth_date']).dt.date

louisville

,animal_id,animal_type,intake_date,intake_type,intake_subtype,color,breed,breed_2,gender,birth_date,intake_condition,intake_repro,outcome_date,outcome_type,outcome_subtype,outcome_condition,outcome_repro,intake_time,outcome_time
4,a281756,dog,2006-09-11,owner sur,otc,white,pit bull terrier,NaN,male,2005-09-11,healthy,fertile,2006-09-12,euth,time/space,healthy,fertile,18:10:00,13:44:00
6,a256128,dog,2005-11-26,stray,field,brown,american pit bull terrier,mix,male,NaT,healthy,fertile,2005-12-08,euth,medical,healthy,fertile,12:35:00,23:59:00
8,a316619,dog,2007-06-29,stray,field,white,labrador retriever,mix,male,NaT,healthy,fertile,2007-07-04,euth,time/space,healthy,fertile,20:10:00,13:12:00
15,a319056,dog,2007-07-19,stray,otc,tricolor,beagle,mix,neutered male,NaT,healthy,altered,2007-08-07,euth,time/space,healthy,altered,22:32:00,12:13:00
17,a258842,dog,2005-12-21,confiscate,neglect,white,pit bull terrier,NaN,male,NaT,healthy,fertile,2005-12-29,euth,breed,healthy,fertile,14:30:00,11:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150835,a489623,dog,2013-04-17,stray,field,apricot,dandie dinmont terrier,mix,neutered male,2010-04-17,healthy,fertile,2013-05-08,adoption,internet,healthy,altered,23:01:00,16:19:00
150836,a493152,dog,2013-06-04,stray,field,black,dachshund - wirehaired,NaN,spayed female,2010-06-04,healthy,fertile,2013-06-16,adoption,web pf,healthy,altered,11:02:00,12:55:00
150837,a591486,dog,2016-06-02,owner sur,otc,yellow brindle,greyhound,mix,neutered male,2014-12-02,healthy,fertile,2016-06-25,adoption,friend,healthy,altered,14:48:00,16:22:00
150838,a523743,dog,2014-04-16,stray,otc,cream,cairn terrier,NaN,spayed female,2013-09-16,healthy,fertile,2014-04-23,adoption,internet,healthy,altered,12:05:00,15:11:00


### Dallas Dataframe Formatting

In [ ]:
dallas1718 = dallas17.append(dallas18)
dallas1718.rename(columns={'Animal_Id':'Animal Id',
                           'Animal_Type':'Animal Type',
                           'Animal_Breed':'Animal Breed',
                           'Kennel_Number':'Kennel Number',
                           'Kennel_Status':'Kennel Status',
                           'Tag_Type':'Tag Type',
                           'Activity_Number':'Activity Number',
                           'Activity_Sequence':'Activity Sequence',
                           'Source_Id':'Source Id',
                           'Census_Tract':'Census Tract',
                           'Council_District':'Council District',
                           'Intake_Type':'Intake Type',
                           'Intake_Subtype':'Intake Subtype',
                           'Intake_Total':'Intake Total',
                           'Staff_Id':'Staff Id',
                           'Intake_Date':'Intake Date',
                           'Intake_Time':'Intake Time',
                           'Due_Out':'Due Out',
                           'Intake_Condition':'Intake Condition',
                           'Hold_Request':'Hold Request',
                           'Outcome_Type':'Outcome Type',
                           'Outcome_Subtype':'Outcome Subtype',
                           'Outcome_Date':'Outcome Date',
                           'Outcome_Time':'Outcome Time',
                           'Receipt_Number':'Receipt Number',
                           'Impound_Number':'Impound Number',
                           'Service_Request_Number':'Service Request Number',
                           'Outcome_Condition':'Outcome Condition',
                           'Chip_Status':'Chip Status',
                           'Animal_Origin':'Animal Origin',
                           'Additional_Information':'Additional Information'
                          }, inplace=True)

dallas16.rename(columns={
                        'Animal ID':'Animal Id',
                        'Staff ID':'Staff Id',
                        'Source ID':'Source Id'
                        }, inplace=True)

In [ ]:
dallas = dallas14.append([dallas15,dallas16,dallas1718, dallas19, dallas20])
dallas.shape

In [ ]:
dallas.drop(['Kennel Number','Kennel Status','Tag Type',
             'Activity Number','Activity Sequence','Source Id',
             'Census Tract', 'Council District', 'Intake Total', 'Reason', 'Staff Id',
            'Due Out', 'Hold Request','Receipt Number', 'Impound Number',
            'Service Request Number', 'Chip Status', 'Animal Origin',
            'Additional Information'], axis=1, inplace=True)

In [ ]:
dallas.rename(columns={'Animal Id':'animal_id',
                       'Animal Type':'animal_type',
                       'Animal Breed':'breed',
                       'Intake Type':'intake_type',
                       'Intake Subtype':'intake_subtype',
                       'Intake Date':'intake_date',
                       'Intake Time':'intake_time',
                       'Intake Condition':'intake_condition',
                       'Outcome Type':'outcome_type',
                       'Outcome Date':'outcome_date',
                       'Outcome Time':'outcome_time',
                       'Outcome Condition':'outcome_condition',
                       'Outcome Subtype':'outcome_subtype',
                       'Month':'month',
                       'Year':'year'
    
}, inplace=True)
dallas.head()